# Create LM

In [1]:
import httpx
from typing import List, Optional
import brokit as bk
class Llama(bk.LM):
    def __init__(self, model_name: str, base_url:str = "http://localhost:11434", temperature:float=0.0, top_p:float=1.0, seed:int=55, **kwargs):
        super().__init__(model_name=model_name, model_type=bk.ModelType.CHAT)
        self.base_url = base_url
        self.client = httpx.Client(timeout=60.0)  # Reusable client
        self.model_params = {
            "temperature": temperature,
            "top_p": top_p,
            "seed": seed,
            **kwargs
        }

    def request(self, prompt:Optional[str]=None, messages:Optional[List[bk.Message]]=None, images:Optional[List[bk.Image]]=None, audios=None, **kwargs) -> dict:
        url = f"{self.base_url}/api/chat"
        params = {**self.model_params, **kwargs}
        if messages is not None:
            _messages = [msg.to_dict() if isinstance(msg, bk.Message) else msg for msg in messages]
        else:
            _messages = [{"role": "user", "content": prompt}]
        if images:
            _messages[-1]["images"] = [image.to_base64() for image in images]
        request_params = {
            "model": self.model_name,
            "messages": _messages,
            "stream": False,
            "options": {**params},
        }
        response = self.client.post(
            url,
            json=request_params
        )                
        return response.json()

    def response(self, original_response: dict) -> bk.ModelResponse:
        message = original_response["message"]
        input_tokens = original_response.get("prompt_eval_count", 0)
        output_tokens = original_response.get("eval_count", 0)
        return bk.ModelResponse(
            model_name=self.model_name,
            model_type=self.model_type,
            response=message["content"],
            usage=bk.Usage(input_tokens=input_tokens, output_tokens=output_tokens),
            metadata=None
        )

# Pure QA

In [2]:
class QA(bk.Prompt):
    """Answer a question in supportive manner"""
    question:str = bk.InputField(description="a user's question")
    answer:str = bk.OutputField(description="answer to the question")

llama = Llama(model_name="gemma3:12b")
qa = bk.Predictor(prompt=QA, lm=llama)
response = qa(question="What is the capital of France")
response

Prediction(
    answer="The capital of France is Paris. It's a beautiful city known for its iconic landmarks like the Eiffel Tower and the Louvre Museum! Do you want to know more about Paris or perhaps another French city?"
)

# ChainOfThoughts QA

In [3]:
class COT_QA(bk.Prompt):
    """Answer a question in supportive manner"""
    question:str = bk.InputField(description="a user's question")
    reasoning:str = bk.OutputField(description="reasons why you answer this way")
    answer:str = bk.OutputField(description="answer to the question")

llama = Llama(model_name="gemma3:12b")
cot_qa = bk.Predictor(prompt=COT_QA, lm=llama)
response = cot_qa(question="What is the capital of France?")
response

Prediction(
    reasoning='The capital of France is a well-known fact, often taught in elementary school geography. France is a country in Europe, and its capital city serves as its center of government and a major cultural hub.',
    answer='The capital of France is Paris.'
)

In [4]:
llama.history

[LMHistory(id='37691832-db8c-477a-9bc8-2ef63cd46688', timestamp=datetime.datetime(2026, 2, 7, 14, 43, 6, 235960), error=None, model_name='gemma3:12b', model_type='chat', request=[{'role': 'system', 'content': "Your input fields are:\n1. question (<class 'str'>): a user's question\nYour output fields are:\n1. reasoning (<class 'str'>): reasons why you answer this way\n2. answer (<class 'str'>): answer to the question\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||reasoning||>\n{reasoning}\n\n<||answer||>\n{answer}\n\n<||completed||>\nIn adhering to this structure, your objective is: \nAnswer a question in supportive manner"}, {'role': 'user', 'content': '<||question||>\nWhat is the capital of France?\n\nRespond with the corresponding output fields, starting with the field: `<||reasoning||>`, `<||answer||>` and then ending with the marker for `<||completed||>`.'}], response='<||reasoning||>\nThe capi

In [5]:
cot_qa.history

[PredictorHistory(id='3a8989f6-5d3c-458d-9bc5-78300b665938', timestamp=datetime.datetime(2026, 2, 7, 14, 43, 6, 236682), error=None, predictor_name='COT_QA', inputs=[{'role': 'system', 'content': "Your input fields are:\n1. question (<class 'str'>): a user's question\nYour output fields are:\n1. reasoning (<class 'str'>): reasons why you answer this way\n2. answer (<class 'str'>): answer to the question\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||reasoning||>\n{reasoning}\n\n<||answer||>\n{answer}\n\n<||completed||>\nIn adhering to this structure, your objective is: \nAnswer a question in supportive manner"}, {'role': 'user', 'content': '<||question||>\nWhat is the capital of France?\n\nRespond with the corresponding output fields, starting with the field: `<||reasoning||>`, `<||answer||>` and then ending with the marker for `<||completed||>`.'}], outputs={'reasoning': 'The capital of France is 